## Setup

In [33]:
pip install -q datasets cohere openai transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.3 MB/s eta 0:00:00


In [5]:
COHERE_API_KEY = "COHERE_API_KEY"
OPENAI_API_KEY = "OPENAI_API_KEY"

## Load Dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset("joelito/plain_english_contracts_summarization", split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/joelito___json/joelito--plain_english_contracts_summarization-25f5156f2d2d542c/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


## Prepare sample legal document for summarization task

In [3]:
pokemon_go_servie_terms = ''

for i in range(0,len(dataset)):
  if dataset[i]['doc'] == 'Pokemon GO Terms of Service':
    pokemon_go_servie_terms += dataset[i]['original_text']
    print(dataset[i]['original_text'])

welcome to the pokémon go video game services which are accessible via the niantic inc niantic mobile device application the app. to make these pokémon go terms of service the terms easier to read our video game services the app and our websites located at http pokemongo nianticlabs com and http www pokemongolive com the site are collectively called the services. please read carefully these terms our trainer guidelines and our privacy policy because they govern your use of our services.
by using our services you are agreeing to these terms our trainer guidelines and our privacy policy. if you are the parent or legal guardian of a child under the age of 13 the parent you are agreeing to these terms on behalf of yourself and your child ren who are authorized to use the services pursuant to these terms and in our privacy policy. if you don t agree to these terms our trainer guidelines and our privacy policy do not use the services.
if you want to use certain features of the services you w

## Using Cohere summarize API

In [6]:
import cohere

cohere_client = cohere.Client(COHERE_API_KEY)
cohere_client

In [7]:
response = cohere_client.summarize(
    text=pokemon_go_servie_terms,
    model='summarize-xlarge',
    length='medium',
    format='bullets',
    extractiveness='high',
    temperature=0.3,
    additional_command=None,
)

In [8]:
response.summary

'-This Pokemon Go Terms of Service (including the arbitration agreement) is a contract between you and Niantic, Inc.\n- ("Niantic").\n- It applies to your use of the Services and the other products and services offered by Niantic (collectively, the "Services").\n- If you do not agree to these terms, you must not use the Services.\n- By making any user content available through the Services you grant Niantic a nonexclusive, perpetual, irrevocable, transferable, sublicensable, worldwide royalty-free license to use, copy, modify, create derivative works based upon, publicly display, publicly perform and distribute your user content in connection with operating and providing the Services and content to you and to other account holders.'

## Using OpenAI 

In [14]:
WandB_confidentiality_terms = "a. Each party that receives (“Receiving Party”) Confidential Information of the other party (“Disclosing Party”) will protect and preserve such Confidential Information as confidential, using no less care than that with which it protects and preserves its own confidential and proprietary information (but in no event less than a reasonable degree of care), and will not use or disclose the Confidential Information for any purpose except to perform its obligations and exercise its rights under this Agreement and applicable Order Forms. b. Receiving Party may disclose, distribute, or disseminate Disclosing Party’s Confidential Information to any of its officers, directors, members, managers, partners, employees, contractors, or agents (its “Representatives”), provided Receiving Party reasonably believes that its Representatives have a need to know and such Representatives are bound by confidentiality obligations at least as restrictive as those contained herein. The Receiving Party will at all times remain responsible for any violations of this Agreement by any of its Representatives. c. A Receiving Party will not violate its confidentiality obligations if it discloses Disclosing Party’s Confidential Information if required by applicable laws, including by court subpoena or similar instrument so long as the Receiving Party provides the Disclosing Party with written notice of the required disclosure so as to allow the Disclosing Party to contest or seek to limit the disclosure or obtain a protective order. If no protective order or other remedy is obtained, the Receiving Party will furnish only that portion of the Confidential Information that is legally required, and agrees to exercise reasonable efforts to ensure that confidential treatment will be accorded to the Confidential Information so disclosed."

In [25]:
import openai

openai.api_key = OPENAI_API_KEY

summarization_prompt = 'Summarize the following text: '+ WandB_confidentiality_terms

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "user", "content": summarization_prompt }
]
)

##MAX_TOKEN_LENGTH = 4097 with gpt-3.5-turbo

In [26]:
## Summary result

response['choices'][0]['message']['content']

'\n\nThe parties that receive confidential information from each other must protect it and only use it for the purposes outlined in their agreement. The receiving party may disclose the confidential information to its representatives, provided they are bound by similar confidentiality obligations. If required by law to disclose the information, the receiving party must notify the disclosing party and make reasonable efforts to limit the disclosure and ensure the information is treated as confidential.'

In [27]:
## Using GPT-3 (davinci) variant -> MAX TOKENS = 2049

response = openai.Completion.create(
  model="text-davinci-003",
  prompt=summarization_prompt,
  temperature=0.3
)

In [30]:
response['choices'][0]['text']

'\n\nThe Receiving Party must protect the Confidential Information of the Discl'

## Using Pegasus fine-tuned on legal data (IN-Abs dataset) 



In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-pegasus")  
model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-pegasus")

In [3]:
text = """On March 5, 2021, the Securities and Exchange Commission charged AT&T, Inc. with repeatedly violating Regulation FD, and three of its Investor Relations executives with aiding and abetting AT&T's violations, by selectively disclosing material nonpublic information to research analysts. According to the SEC's complaint, AT&T learned in March 2016 that a steeper-than-expected decline in its first quarter smartphone sales would cause AT&T's revenue to fall short of analysts' estimates for the quarter. The complaint alleges that to avoid falling short of the consensus revenue estimate for the third consecutive quarter, AT&T Investor Relations executives Christopher Womack, Michael Black, and Kent Evans made private, one-on-one phone calls to analysts at approximately 20 separate firms. On these calls, the AT&T executives allegedly disclosed AT&T's internal smartphone sales data and the impact of that data on internal revenue metrics, despite the fact that internal documents specifically informed Investor Relations personnel that AT&T's revenue and sales of smartphones were types of information generally considered "material" to AT&T investors, and therefore prohibited from selective disclosure under Regulation FD. The complaint further alleges that as a result of what they were told on these calls, the analysts substantially reduced their revenue forecasts, leading to the overall consensus revenue estimate falling to just below the level that AT&T ultimately reported to the public on April 26, 2016. The SEC's complaint, filed in federal district court in Manhattan, charges AT&T with violations of the disclosure provisions of Section 13(a) of the Securities Exchange Act of 1934 and Regulation FD thereunder, and charges Womack, Evans and Black with aiding and abetting these violations. The complaint seeks permanent injunctive relief and civil monetary penalties against each defendant. The SEC's investigation was conducted by George N. Stepaniuk, Thomas Peirce, and David Zetlin-Jones of the SEC's New York Regional Office. The SEC's litigation will be conducted by Alexander M. Vasilescu, Victor Suthammanont, and Mr. Zetlin-Jones. The case is being supervised by Sanjay Wadhwa."""

input_tokenized = tokenizer.encode(text, return_tensors='pt',max_length=1024,truncation=True)
summary_ids = model.generate(input_tokenized,
                                  num_beams=9,
                                  no_repeat_ngram_size=3,
                                  length_penalty=2.0,
                                  min_length=50,
                                  max_length=100,
                                  early_stopping=True)
summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]

In [4]:
summary

"On March 5, 2021, the Securities and Exchange Commission charged AT&T, Inc. and three of its Investor Relations executives with aiding and abetting the company's violations of the antifraud provisions of Section 10(b) of the Securities Exchange Act of 1934 and Rule 10b-5 thereunder. According to the SEC's complaint, the company learned in March 2016 that a steeper-than-expected decline in its first quarter smartphone sales would cause its revenue to fall short of analysts'"